In [ ]:
!pip install plotly

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, layers, Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/fraud detection/creditcard.csv')

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283726 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    283726 non-null  float64
 1   V1      283726 non-null  float64
 2   V2      283726 non-null  float64
 3   V3      283726 non-null  float64
 4   V4      283726 non-null  float64
 5   V5      283726 non-null  float64
 6   V6      283726 non-null  float64
 7   V7      283726 non-null  float64
 8   V8      283726 non-null  float64
 9   V9      283726 non-null  float64
 10  V10     283726 non-null  float64
 11  V11     283726 non-null  float64
 12  V12     283726 non-null  float64
 13  V13     283726 non-null  float64
 14  V14     283726 non-null  float64
 15  V15     283726 non-null  float64
 16  V16     283726 non-null  float64
 17  V17     283726 non-null  float64
 18  V18     283726 non-null  float64
 19  V19     283726 non-null  float64
 20  V20     283726 non-null  float64
 21  V21     28

In [8]:
missing_values = df.isna().sum().to_frame()
missing_values = missing_values.rename(columns= {0: 'missing_values'})
missing_values['% of total'] = (missing_values['missing_values'] / df.shape[0]).round(2)
missing_values

,missing_values,% of total
Time,0,0.0
V1,0,0.0
V2,0,0.0
V3,0,0.0
V4,0,0.0
V5,0,0.0
V6,0,0.0
V7,0,0.0
V8,0,0.0
V9,0,0.0


In [9]:
df['Class'].value_counts()

0    283253
1       473
Name: Class, dtype: int64

In [10]:
fraud = df[df['Class'] == 1]
valid = df[df['Class'] == 0]

In [12]:
print('There are', len(fraud), 'fraudulent')
print('There are', len(valid), 'Valid Transaction')

print('There are', f'{len(fraud)/len(df) * 100 :.2f} ',  '% of fraudulent transaction in the dataframe')


There are 473 fraudulent
There are 283253 Valid Transaction
There are 0.17  % of fraudulent transaction in the dataframe


In [14]:
# visualise class representation in the data set
fraud_or_valid = df['Class'].value_counts().tolist()
labels = ['Genuine', 'Fraudulent']
fig = px.pie(values=fraud_or_valid, names=labels, 
            title='Genuine vs Fraudulent Transactions')
fig.show()

There are very low numbers of fraudulent transaction in the data compared to the number of valid data. Making the dataset highly imbalanced resulting in the model not being able to learn much from the miniority class. we can overcome this problem by using class weight

In [15]:
#slice dataset to x and y

x = df.drop(['Class', 'Time'], axis=1)
y = df['Class']

In [16]:
#split dataset into training and test
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size = 0.3, random_state = 123)

In [17]:
#compute class weight to correct imbalance during training

weight = compute_class_weight('balanced', classes = np.unique(y_train), y=y_train)
class_weight = {0: weight[0],
                1: weight[1]
                }
print(class_weight)

{0: 0.5008346908617742, 1: 300.012084592145}


In [18]:
#scale data using standardscaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [96]:
#create a simple sequential model with 2 dense layer

model = Sequential(layers=[
    layers.Input(shape=(29,), dtype='float32'),
    layers.Dense(256, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(2, activation='sigmoid')
]
)

In [97]:
EPOCHS =500
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)


In [98]:
model.compile(optimizer=optimizer,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics='accuracy')

In [99]:
history = model.fit(
          x_train_scaled, y_train, epochs=EPOCHS, 
          batch_size=4096, 
          validation_split=0.2,
          class_weight=class_weight
          )

Epoch 1/500
39/39 [==============================] - 1s 10ms/step - loss: 0.3660 - accuracy: 0.8953 - val_loss: 0.3458 - val_accuracy: 0.9857
Epoch 2/500
39/39 [==============================] - 0s 6ms/step - loss: 0.2435 - accuracy: 0.9849 - val_loss: 0.2431 - val_accuracy: 0.9859
Epoch 3/500
39/39 [==============================] - 0s 6ms/step - loss: 0.1945 - accuracy: 0.9876 - val_loss: 0.1932 - val_accuracy: 0.9850
Epoch 4/500
39/39 [==============================] - 0s 6ms/step - loss: 0.1644 - accuracy: 0.9864 - val_loss: 0.1519 - val_accuracy: 0.9882
Epoch 5/500
39/39 [==============================] - 0s 6ms/step - loss: 0.1460 - accuracy: 0.9898 - val_loss: 0.1360 - val_accuracy: 0.9869
Epoch 6/500
39/39 [==============================] - 0s 6ms/step - loss: 0.1326 - accuracy: 0.9883 - val_loss: 0.1233 - val_accuracy: 0.9864
Epoch 7/500
39/39 [==============================] - 0s 6ms/step - loss: 0.1225 - accuracy: 0.9855 - val_loss: 0.1027 - val_accuracy: 0.9884
Epoch 8/500


In [145]:
#evaluate model using test dataset
loss, accuracy = model.evaluate(x_test_scaled, y_test, batch_size=4096)
print('Model Evaluation Accuracy is', f'{accuracy *100:.1f}' , '%')


#making training and validation plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

epoch_range = np.array(range(EPOCHS))


fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=epoch_range, y=history.history['accuracy'], name="Training Accuracy"),

    row=1, col=1
)
fig.add_trace(
   go.Scatter(x=epoch_range, y=history.history['val_accuracy'], name='Validation Acccuracy'),

    row=1, col=1
)


fig.add_trace(
    go.Scatter(x=epoch_range, y=history.history['loss'], name='Training Loss'),

    row=1, col=2
)
fig.add_trace(
   go.Scatter(x=epoch_range, y=history.history['val_loss'], name='Validation Loss'),

    row=1, col=2
)

fig.update_layout( title_text="Training and Validation Plot")
fig.show()

21/21 [==============================] - 0s 3ms/step - loss: 0.0135 - accuracy: 0.9993
Model Evaluation Accuracy is 99.9 %
